In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from sklearn.model_selection import train_test_split
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score, confusion_matrix
import seaborn as sn
import pandas as pd
import numpy as np
import os
import torch
from torch.utils.data import Dataset, DataLoader


In [14]:
data_dir = '/content/drive/MyDrive/UCI HAR Dataset'
output_dir = '/content/uci_data'
# Dataset/UCI HAR Dataset/data/train/Inertial Signals/body_acc_x_train.txt
# subject_data = np.loadtxt(f'{data_dir}/train/subject_train.txt')
# Samples
train_acc_x = np.loadtxt(f'{data_dir}/train/Inertial Signals/body_acc_x_train.txt')
train_acc_y = np.loadtxt(f'{data_dir}/train/Inertial Signals/body_acc_y_train.txt')
train_acc_z = np.loadtxt(f'{data_dir}/train/Inertial Signals/body_acc_z_train.txt')
train_gyro_x = np.loadtxt(f'{data_dir}/train/Inertial Signals/body_gyro_x_train.txt')
train_gyro_y = np.loadtxt(f'{data_dir}/train/Inertial Signals/body_gyro_y_train.txt')
train_gyro_z = np.loadtxt(f'{data_dir}/train/Inertial Signals/body_gyro_z_train.txt')
train_tot_acc_x = np.loadtxt(f'{data_dir}/train/Inertial Signals/total_acc_x_train.txt')
train_tot_acc_y = np.loadtxt(f'{data_dir}/train/Inertial Signals/total_acc_y_train.txt')
train_tot_acc_z = np.loadtxt(f'{data_dir}/train/Inertial Signals/total_acc_z_train.txt')

test_acc_x = np.loadtxt(f'{data_dir}/test/Inertial Signals/body_acc_x_test.txt')
test_acc_y = np.loadtxt(f'{data_dir}/test/Inertial Signals/body_acc_y_test.txt')
test_acc_z = np.loadtxt(f'{data_dir}/test/Inertial Signals/body_acc_z_test.txt')
test_gyro_x = np.loadtxt(f'{data_dir}/test/Inertial Signals/body_gyro_x_test.txt')
test_gyro_y = np.loadtxt(f'{data_dir}/test/Inertial Signals/body_gyro_y_test.txt')
test_gyro_z = np.loadtxt(f'{data_dir}/test/Inertial Signals/body_gyro_z_test.txt')
test_tot_acc_x = np.loadtxt(f'{data_dir}/test/Inertial Signals/total_acc_x_test.txt')
test_tot_acc_y = np.loadtxt(f'{data_dir}/test/Inertial Signals/total_acc_y_test.txt')
test_tot_acc_z = np.loadtxt(f'{data_dir}/test/Inertial Signals/total_acc_z_test.txt')

# Stacking channels together data
train_data = np.stack((train_acc_x, train_acc_y, train_acc_z,
                       train_gyro_x, train_gyro_y, train_gyro_z,
                       train_tot_acc_x, train_tot_acc_y, train_tot_acc_z), axis=1)
X_test = np.stack((test_acc_x, test_acc_y, test_acc_z,
                      test_gyro_x, test_gyro_y, test_gyro_z,
                      test_tot_acc_x, test_tot_acc_y, test_tot_acc_z), axis=1)
# labels
train_labels = np.loadtxt(f'{data_dir}/train/y_train.txt')
train_labels -= np.min(train_labels)
y_test = np.loadtxt(f'{data_dir}/test/y_test.txt')
y_test -= np.min(y_test)
print(train_data.shape, X_test.shape)
print(train_labels.shape, y_test.shape)

X_train, X_val, y_train, y_val = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

dat_dict = dict()
dat_dict["samples"] = torch.from_numpy(X_train)
dat_dict["labels"] = torch.from_numpy(y_train)
torch.save(dat_dict, os.path.join(output_dir, "train.pt"))

dat_dict = dict()
dat_dict["samples"] = torch.from_numpy(X_val)
dat_dict["labels"] = torch.from_numpy(y_val)
torch.save(dat_dict, os.path.join(output_dir, "val.pt"))

dat_dict = dict()
dat_dict["samples"] = torch.from_numpy(X_test)
dat_dict["labels"] = torch.from_numpy(y_test)
torch.save(dat_dict, os.path.join(output_dir, "test.pt"))

(7352, 9, 128) (2947, 9, 128)
(7352,) (2947,)


In [16]:
data_dir = "/content/uci_data"  # Adjust this based on the correct directory path
output_dir = "./output_data"  # Adjust this based on your desired output directory
os.makedirs(output_dir, exist_ok=True)

few_lbl_percentages = [1, 5, 10, 50, 75]

for percentage in few_lbl_percentages:
    data = torch.load(os.path.join(data_dir, "train_100per.pt"))  # Load from the correct path
    x_data = data["samples"].numpy()
    y_data = data["labels"].numpy()

    X_train, X_val, y_train, y_val = train_test_split(x_data, y_data, test_size=percentage / 100, random_state=0)

    few_shot_dataset = {"samples": X_val, "labels": y_val}

    # Saving data
    data_save = dict()
    data_save["samples"] = torch.from_numpy(X_val)
    data_save["labels"] = torch.from_numpy(y_val)
    torch.save(data_save, os.path.join(output_dir, f"train_{percentage}per.pt"))


In [18]:
def format_data_x(datafile):
    x_data = None
    for item in datafile:
        item_data = np.loadtxt(item, dtype=np.float)
        if x_data is None:
            x_data = np.zeros((len(item_data), 1))
        x_data = np.hstack((x_data, item_data))
    x_data = x_data[:, 1:]
    print("1", x_data.shape) #(7352, 1152)
    X = None
    for i in range(len(x_data)):
        row = np.asarray(x_data[i, :])
        # row = row.reshape(9, 128).T
        row = row.reshape(9, 128).T
        if X is None:
            # X = np.zeros((len(x_data), 128, 9))
            X = np.zeros((len(x_data), 128, 9))
        X[i] = row
    print("1", X.shape) #(7352, 128, 3)
    return X


def format_data_y(datafile):
    data = np.loadtxt(datafile, dtype=np.int) - 1
    YY = np.eye(6)[data]
    return YY


# Load data function, if there exists parsed data file, then use it
# If not, parse the original dataset from scratch
def load_data(data_folder):

    if os.path.isfile(data_folder + 'train_100per.pt') == True:
        print(".pt loaded")
        train_data = torch.load(data_folder + 'train_100per.pt')
        test_data = torch.load(data_folder + 'test.pt')
        X_train = train_data['samples']
        Y_train = train_data['labels']
        X_test = test_data['samples']
        Y_test = test_data['labels']
        return X_train, Y_train, X_test, Y_test
    else:
        # This for processing the dataset from scratch
        # After downloading the dataset, put it to somewhere that str_folder can find
        str_folder = data_folder
        INPUT_SIGNAL_TYPES = [
            "body_acc_x_",
            "body_acc_y_",
            "body_acc_z_",
            "body_gyro_x_",
            "body_gyro_y_",
            "body_gyro_z_",
            "total_acc_x_",
            "total_acc_y_",
            "total_acc_z_"
        ]

        str_train_files = [str_folder + 'train/' + 'Inertial Signals/' + item + 'train.txt' for item in
                           INPUT_SIGNAL_TYPES]
        str_test_files = [str_folder + 'test/' + 'Inertial Signals/' +
                          item + 'test.txt' for item in INPUT_SIGNAL_TYPES]
        str_train_y = str_folder + 'train/y_train.txt'
        str_test_y = str_folder + 'test/y_test.txt'

        X_train = format_data_x(str_train_files)
        X_test = format_data_x(str_test_files)
        Y_train = format_data_y(str_train_y)
        Y_test = format_data_y(str_test_y)

        print(str_folder)
    return X_train, onehot_to_label(Y_train), X_test, onehot_to_label(Y_test)




def onehot_to_label(y_onehot):
    a = np.argwhere(y_onehot == 1)
    return a[:, -1]


class data_loader(Dataset):
    def __init__(self, samples, labels, t):
        self.samples = samples
        self.labels = labels
        self.T = t

    def __getitem__(self, index):
        sample, target = self.samples[index], self.labels[index]
        if self.T:
            return self.T(sample), target
        else:
            return sample, target

    def __len__(self):
        return len(self.samples)


def normalize(x):
    x_min = x.min(axis=(0, 2, 3), keepdims=True)
    x_max = x.max(axis=(0, 2, 3), keepdims=True)
    x_norm = (x - x_min) / (x_max - x_min)
    return x_norm


def load(data_folder, batch_size):
    x_train, y_train, x_test, y_test = load_data(data_folder)
    transform = None
    train_set = data_loader(x_train, y_train, transform)
    test_set = data_loader(x_test, y_test, transform)
    train_loader = DataLoader(
        train_set, batch_size=batch_size, shuffle=True, drop_last=True)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader


# data_folder = '/Dataset/'
# load(data_folder, 64)

In [ ]:
"""
Attention blocks
Reference: Learn To Pay Attention
"""
class ProjectorBlock(nn.Module):
    def __init__(self, in_features, out_features):
        super(ProjectorBlock, self).__init__()
        self.op = nn.Conv1d(in_channels=in_features, out_channels=out_features,
            kernel_size=1, padding=0, bias=False)

    def forward(self, x):
        return self.op(x)


class SpatialAttn(nn.Module):
    def __init__(self, in_features, normalize_attn=True):
        super(SpatialAttn, self).__init__()
        self.normalize_attn = normalize_attn
        self.op = nn.Conv2d(in_channels=in_features, out_channels=1,
            kernel_size=1, padding=0, bias=False)

    def forward(self, l, g):
        N, C, H, W = l.size()
        c = self.op(l+g) # (batch_size,1,H,W)
        if self.normalize_attn:
            a = F.softmax(c.view(N,1,-1), dim=2).view(N,1,H,W)
        else:
            a = torch.sigmoid(c)
        g = torch.mul(a.expand_as(l), l)
        if self.normalize_attn:
            g = g.view(N,C,-1).sum(dim=2) # (batch_size,C)
        else:
            g = F.adaptive_avg_pool2d(g, (1,1)).view(N,C)
        return c.view(N,1,H,W), g

"""
Temporal attention block
Reference: https://github.com/philipperemy/keras-attention-mechanism
"""
class TemporalAttn(nn.Module):
    def __init__(self, hidden_size):
        super(TemporalAttn, self).__init__()
        self.hidden_size = hidden_size
        self.fc1 = nn.Linear(self.hidden_size, self.hidden_size, bias=False)
        self.fc2 = nn.Linear(self.hidden_size*2, self.hidden_size, bias=False)

    def forward(self, hidden_states):
        # (batch_size, time_steps, hidden_size)
        score_first_part = self.fc1(hidden_states)
        # (batch_size, hidden_size)
        h_t = hidden_states[:,-1,:]
        # (batch_size, time_steps)
        score = torch.bmm(score_first_part, h_t.unsqueeze(2)).squeeze(2)
        attention_weights = F.softmax(score, dim=1)
        # (batch_size, hidden_size)
        # context_vector = torch.bmm(hidden_states.permute(0,2,1), attention_weights.unsqueeze(2)).squeeze(2)\
        context_vector = torch.bmm(hidden_states.transpose(1, 2), attention_weights.unsqueeze(2)).squeeze(2)
        # (batch_size, hidden_size*2)
        pre_activation = torch.cat((context_vector, h_t), dim=1)
        # (batch_size, hidden_size)
        attention_vector = self.fc2(pre_activation)
        attention_vector = torch.tanh(attention_vector)

        return attention_vector, attention_weights

# Test
if __name__ == '__main__':
    # 2d block
    # spatial_block = SpatialAttn(in_features=3)
    # l = torch.randn(16, 3, 128, 128)
    # g = torch.randn(16, 3, 128, 128)
    # print(spatial_block(l, g))
    # temporal block
    temporal_block = TemporalAttn(hidden_size=256)
    x = torch.randn(16, 30, 256)
    attention_vector, attention_weights = temporal_block(x)
    # print(attention_vector.shape) # should output (16, 256)
    # print(attention_weights.shape) # should output (16, 30)
    # print(temporal_block(x).shape)

In [19]:
class Network(nn.Module):
    
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels = 9, out_channels = 64, kernel_size=6, stride=1, padding=2),            
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(in_channels = 64, out_channels = 128, kernel_size=3, stride=1, padding =2),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )

        self.flatten = torch.nn.Flatten()
        self.dropout = torch.nn.Dropout(0.1)

        self.lstm = nn.LSTM(input_size=32, hidden_size=128, num_layers=1)
        self.tanh = torch.nn.Tanh()
        self.attn = TemporalAttn(hidden_size=128)
        self.fc = nn.Linear(in_features=128, out_features=6)
        self.softmax = torch.nn.Softmax(dim = 1)


    def forward(self, x):
        # print("1", x.dtype, x.shape)
        # x = x.permute(0,2,1)
        out = self.conv1(x)
        # print("c1", out.dtype, out.shape)
        out = self.conv2(out)
        # print("c2", out.dtype, out.shape)
        out = self.dropout(out)
        # print("dropout", out.shape, out.dtype)
        out, hidden = self.lstm(out)
        out = self.tanh(out)
        # print("lstm", out.dtype, out.shape)
        #attention
        out, weights=self.attn(out)
        out = self.flatten(out)
        # print("flatten", out.dtype, out.shape)
        out = self.fc(out)
        # print("fc", out.dtype, out.shape)
        out = self.softmax(out)
        # print("sm", out.dtype, out.shape)

        return out

In [20]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
data_folder = './uci_data/'
result = []
f1_result = []
classes = ['WALKING', 'WALKING_UPSTAIRS',  'WALKING_DOWNSTAIRS','SITTING', 'STANDING', 'LAYING']

result_path='//output//'
testAcc_csv=result_path+'result_cnn-lstm_UCI.csv'
f1_csv=result_path+'result_f1_cnn-lstm_UCI.csv'
confusion_img=result_path+'confusion matrix_cnn-lstm_UCI.png'
plot_img=result_path+'plot_cnn-lstm_UCI.png'

In [21]:
def load_data_interface():
    train_loader, test_loader = load(
        data_folder, batch_size=64)  # Ensure batch_size is defined
    return train_loader, test_loader


In [22]:
def calculate_accuracy(predictions, labels):
    correct = (np.array(predictions) == np.array(labels)).sum()
    total = len(predictions)
    return (correct / total) * 100

def calculate_f1_scores(predictions, labels):
    f1_scores = f1_score(labels, predictions, average=None)
    avg_f1_score = np.mean(f1_scores)
    return avg_f1_score

def generate_confusion_matrix(predictions, labels):
    cm = confusion_matrix(labels, predictions)
    return cm

In [23]:
def evaluate_model(model, test_loader):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for samples, labels in test_loader:
            samples, labels = samples.to(DEVICE).float(), labels.to(DEVICE).long()
            outputs = model(samples)
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)

            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = calculate_accuracy(all_predictions, all_labels)
    f1_scores = calculate_f1_scores(all_predictions, all_labels)
    confusion_matrix = generate_confusion_matrix(all_predictions, all_labels)

    np.savetxt(testAcc_csv, np.array([accuracy]), delimiter=',', fmt='%.4f')
    np.savetxt(f1_csv, f1_scores.reshape(1, -1), delimiter=',', fmt='%.4f')
    np.savetxt(confusion_img, confusion_matrix, delimiter=',', fmt='%d')

    return accuracy, f1_scores, confusion_matrix


In [29]:
def train_model(model, optimizer, train_loader, test_loader, n_epochs):
    criterion = nn.CrossEntropyLoss()
    result = []  # To store training results for plotting or analysis

    for epoch in range(n_epochs):
        model.train()
        correct = 0
        total_loss = 0

        for sample, label in train_loader:
            sample, label = sample.to(DEVICE).float(), label.to(DEVICE).long()
            optimizer.zero_grad()
            output = model(sample)
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            correct += (predicted == label).sum().item()

        train_accuracy = 100 * correct / len(train_loader.dataset)
        avg_loss = total_loss / len(train_loader)

        # Validate the model on the test set after each epoch
        test_accuracy, _, _ = evaluate_model(model, test_loader)  # Assuming accuracy is the first value returned

        # Store the epoch-wise results for analysis or plotting
        result.append([train_accuracy, test_accuracy, avg_loss])

        print(f'Epoch [{epoch+1}/{n_epochs}] - '
              f'Train Accuracy: {float(train_accuracy):.2f}% - '
              f'Test Accuracy: {float(test_accuracy):.2f}% - '
              f'Average Loss: {float(avg_loss):.4f}')
        
    torch.save(model.state_dict(), 'trained_model.pth')
    return result  # Return the training results for later use (e.g., plotting)


In [ ]:
train_loader, test_loader = load_data_interface()
model = Network().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
result = train_model(model, optimizer, train_loader, test_loader, 30)
accuracy, f1_scores, confusion_matrix = evaluate_model(model, test_loader, testAcc_csv, f1_csv, confusion_img)
# Calculate averages or other post-processing steps